## UMAP reproducibility benchmark (runtime & trustworthiness)

In [1]:
from umap import UMAP as umap_learn
from cuml.manifold import UMAP as umap_cuml
from cuml.metrics import trustworthiness

from sklearn.datasets import make_blobs
import time
import numpy as np

In [8]:
def generate_data(n_samples):
    X, y = make_blobs(n_samples=n_samples, n_features=args['n_features'],
                      centers=args['centers'], cluster_std=8.0)
    return X

In [9]:
def benchmark_model(model_constr, args, data):
    durations = []
    trust_scores = []
    for i in range(args['n_iter'] + 1):
        # Instantiate model
        model = model_constr(n_neighbors=args['n_neighbors'], n_epochs=args['n_epochs'],
                           random_state=args['random_state'])
        
        # Perform transformation and measure time
        start = time.time()
        transformed = model.fit_transform(data)
        durations.append(time.time()-start)
        
        # Compute trustworthiness score
        trust_scores.append(trustworthiness(data, transformed, n_neighbors=args['n_neighbors']))
    
    durations = np.array(durations[1:])
    trust_scores = np.array(trust_scores)
    
    # Compute runtime average and variance as well as trustworthiness score average
    return durations.mean(), durations.var(), trust_scores.mean()

In [10]:
def benchmark(args):
    for n_samples in args['n_samples']:
        # Generate dataset
        X = generate_data(n_samples)

        # Benchmarks the two models
        print("For dataset of size ({}, {}) :".format(n_samples, args['n_features']))
        
        print("\tWithout random state :")
        args['random_state'] = None
        ul_inconsistent = benchmark_model(umap_learn, args, X)
        cuml_inconsistent = benchmark_model(umap_cuml, args, X)
        print_results(ul_inconsistent, cuml_inconsistent)
        
        print("\tWith random state :")
        args['random_state'] = 42
        ul_consistent = benchmark_model(umap_learn, args, X)
        cuml_consistent = benchmark_model(umap_cuml, args, X)
        print_results(ul_consistent, cuml_consistent)
        
        a = cuml_consistent[0]
        b = cuml_inconsistent[0]
        slowdown = ((a - b) / a) * 100
        print('\tcuML consistent pathway is {:.2f}% slower\n'.format(slowdown))

In [11]:
def print_results(ul_scores, cuml_scores):
    ul_dur_mean, ul_dur_var, ul_trust = ul_scores
    print("\t\tUMAP-LEARN: runtime avg - var: {:.2f} - {:.2f}, tustworthiness: {:.2f}".format(ul_dur_mean, ul_dur_var, ul_trust))
    cuml_dur_mean, cuml_dur_var, cuml_trust = cuml_scores
    print("\t\tCUML UMAP: runtime avg - var: {:.2f} - {:.2f}, tustworthiness: {:.2f}".format(cuml_dur_mean, cuml_dur_var, cuml_trust))

In [12]:
import warnings
warnings.filterwarnings('ignore')

args = {'n_samples':[1000, 4000, 12000], 'n_features':100, 'centers':500,
        'n_neighbors':15, 'n_epochs':500, 'n_iter': 5}

benchmark(args)

For dataset of size (1000, 100) :
	Without random state :
		UMAP-LEARN: runtime avg - var: 2.56 - 0.00, tustworthiness: 0.60
		CUML UMAP: runtime avg - var: 0.24 - 0.00, tustworthiness: 0.60
	With random state :
		UMAP-LEARN: runtime avg - var: 2.53 - 0.00, tustworthiness: 0.60
		CUML UMAP: runtime avg - var: 0.24 - 0.00, tustworthiness: 0.59
	cuML consistent pathway is 1.64% slower

For dataset of size (4000, 100) :
	Without random state :
		UMAP-LEARN: runtime avg - var: 10.01 - 0.01, tustworthiness: 0.70
		CUML UMAP: runtime avg - var: 0.26 - 0.00, tustworthiness: 0.67
	With random state :
		UMAP-LEARN: runtime avg - var: 10.03 - 0.01, tustworthiness: 0.70
		CUML UMAP: runtime avg - var: 0.29 - 0.00, tustworthiness: 0.66
	cuML consistent pathway is 11.17% slower

For dataset of size (12000, 100) :
	Without random state :
		UMAP-LEARN: runtime avg - var: 33.34 - 0.00, tustworthiness: 0.98
		CUML UMAP: runtime avg - var: 0.28 - 0.00, tustworthiness: 0.95
	With random state :
		UMAP-LE